In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('./0121-政府风险偏好-关键词.xlsx')

In [3]:
df

,词汇,词性,举例,处理,备注
0,实义词,消极,困难、问题、不足、难点、难题、风险、压力、矛盾、危机、瓶颈、制约、短板,只要提到这些消极词汇，均认为这句话是【保守】的，打标签“0”,优先级最高
1,NaN,中性或积极,其他,需要和程度副词以及其他词汇搭配判断,NaN
2,程度副词,高度,非常、很、十分、最、格外、相当、极其、显著、大幅、重大、明显、高位,只要提到这些词汇，均认为这句话是【激进】的，打标签“1”,如果出现矛盾，可标注，后续人工进行判断
3,NaN,轻微,不完全、相对、稍、还、仍、偏、较好,只要提到这些词汇，均认为这句话是【保守】的，打标签“0”,NaN
4,其他,积极,大刀阔斧、雷厉风行、毫不动摇、坚持不懈、强势、全力、发力、力促、力争、争创、大力、着力、倾力...,只要提到这些词汇，均认为这句话是【激进】的，打标签“1”,NaN
5,NaN,消极,紧日子、紧张、不易、艰难、不寻常、不平凡、下行压力、严峻、不稳定、不确定、不充分、不快、不够...,只要提到这些词汇，均认为这句话是【保守】的，打标签“0”,NaN


In [4]:
keywords_up = []
keywords_down = []
for index,row in df.iterrows():
    if '1' in row['处理']:
        for word in row['举例'].split('、'):
            keywords_up.append(word.replace('（','').replace('）',''))
    elif '0' in row['处理']:
        keywords_down += row['举例'].split('、')
keywords_up.append('更')
keywords_up.append('越')
keywords_up.append('快速')

In [7]:
from docx import Document
import os
import re
import pdb
path1 = './地级工作报告(2007-2021)' #待读取文件的文件夹绝对地址
files = os.listdir(path1) #获得文件夹中所有文件的名称列表
result = []
test_data = []
for file in files:
    try:
        if int(file[-9:-5]) >= 2010 and int(file[-9:-5]) <= 2013:
            path = path1 +  '/' + file
            document = Document(path)
            sentences = document.paragraphs[0].text.strip().replace('\n','').split('。')
            
            for sentence in sentences:
                keywords_extract = []
                keywords_extract = []
                labels = []
                words_up = re.findall('|'.join(keywords_up),sentence)
                words_down = re.findall('|'.join(keywords_down),sentence)
                labels.append(str(len(words_up)))
                labels.append(str(len(words_down)))
                result.append([sentence,','.join(labels)])
        else:
            path = path1 +  '/' + file
            document = Document(path)
            sentences = document.paragraphs[0].text.strip().replace('\n','').split('。')
            for sentence in sentences:
                keywords_extract = []
                keywords_extract = []
                labels = []
                words_up = re.findall('|'.join(keywords_up),sentence)
                words_down = re.findall('|'.join(keywords_down),sentence)
                labels.append(str(len(words_up)))
                labels.append(str(len(words_down)))
                test_data.append([sentence,','.join(labels)])
    except:
        pass

In [8]:
results = pd.DataFrame(result,columns=['句子','标签'])
results = results.drop_duplicates().reset_index(drop=True)
train_result = result[ :int(0.9 * len(result))]
dev_result = result[int(0.9 * len(result)):]
train_result_df = pd.DataFrame(train_result,columns=['句子','标签'])
dev_result_df = pd.DataFrame(dev_result,columns=['句子','标签'])
test_data_df = pd.DataFrame(test_data,columns=['句子','标签'])

In [16]:
from sklearn.utils import shuffle
data_dir = './data'
def preprocess_data(data_type):
    train_data_process = []
    dev_data_process = []
    test_data_process = []
    train_data_path =  data_dir + '/train_' + data_type + '.txt'
    dev_data_path = data_dir + '/dev_' + data_type + '.txt'
    test_data_path =  data_dir + '/test_' + data_type + '.txt'
    if data_type == 'model_1':
        for index,row in train_result_df.iterrows():
            if '0' not in row['标签'].split(','):
                continue
            if row['标签'].split(',')[0] == '0':
                train_data_process.append([row['句子'],'0'])
            else:
                train_data_process.append([row['句子'],'1'])
        for index,row in dev_result_df.iterrows():
            if '0' not in row['标签'].split(','):
                continue
            if row['标签'].split(',')[0] == '0':
                dev_data_process.append([row['句子'],'0'])
            else:
                dev_data_process.append([row['句子'],'1'])
        for index,row in test_data_df.iterrows():
            if '0' not in row['标签'].split(','):
                continue
            if row['标签'].split(',')[0] == '0':
#                 pdb.set_trace()
                test_data_process.append([row['句子'],'0'])
            else:
                test_data_process.append([row['句子'],'1'])
        train_data_df = pd.DataFrame(train_data_process,columns=['sentence','label']).drop_duplicates().reset_index(drop=True)
        train_data_df_1 = train_data_df[train_data_df['label']=='1']
        train_data_df_0 = train_data_df[train_data_df['label']=='0']
        print('model_1,标签1:{},标签0:{}'.format(len(train_data_df_1),len(train_data_df_0)))
    if data_type == 'model_2':
        for index,row in train_result_df.iterrows():
            if row['标签'].split(',')[0] > row['标签'].split(',')[1] and row['标签'].split(',')[1] == '0':
                train_data_process.append([row['句子'],'1'])
            else:
                train_data_process.append([row['句子'],'0'])
        for index,row in dev_result_df.iterrows():
            if row['标签'].split(',')[0] > row['标签'].split(',')[1] and row['标签'].split(',')[1] == '0':
                dev_data_process.append([row['句子'],'1'])
            else:
                dev_data_process.append([row['句子'],'0'])
        for index,row in test_data_df.iterrows():
            if row['标签'].split(',')[0] > row['标签'].split(',')[1] and row['标签'].split(',')[1] == '0':
                test_data_process.append([row['句子'],'1'])
            else:
                test_data_process.append([row['句子'],'0'])
        train_data_df = pd.DataFrame(train_data_process,columns=['sentence','label']).drop_duplicates().reset_index(drop=True)
        train_data_df_1 = train_data_df[train_data_df['label']=='1']
        train_data_df_0 = train_data_df[train_data_df['label']=='0']
        print('model_2,标签1:{},标签0:{}'.format(len(train_data_df_1),len(train_data_df_0)))
    if data_type == 'model_3':
        for index,row in train_result_df.iterrows():
            if '0' in row['标签'].split(',') and len(set(row['标签'].split(','))) == 2:
                if row['标签'].split(',')[0] > row['标签'].split(',')[1]:
                    train_data_process.append([row['句子'],'1'])
                else:
                    train_data_process.append([row['句子'],'0'])
            else:
                train_data_process.append([row['句子'],'2'])
        for index,row in dev_result_df.iterrows():
            if '0' in row['标签'].split(',') and len(set(row['标签'].split(','))) == 2:
                if row['标签'].split(',')[0] > row['标签'].split(',')[1]:
                    dev_data_process.append([row['句子'],'1'])
                else:
                    dev_data_process.append([row['句子'],'0'])
            else:
                dev_data_process.append([row['句子'],'2'])
        for index,row in test_data_df.iterrows():
            if '0' in row['标签'].split(',') and len(set(row['标签'].split(','))) == 2:
                if row['标签'].split(',')[0] > row['标签'].split(',')[1]:
                    test_data_process.append([row['句子'],'1'])
                else:
                    test_data_process.append([row['句子'],'0'])
            else:
                test_data_process.append([row['句子'],'2'])
        train_data_df = pd.DataFrame(train_data_process,columns=['sentence','label']).drop_duplicates().reset_index(drop=True)
        train_data_df_0 = train_data_df[train_data_df['label']=='0']
        train_data_df_1 = train_data_df[train_data_df['label']=='1']        
        train_data_df_2 = train_data_df[train_data_df['label']=='2']
        print('model_3, 标签2:{},标签1:{},标签0:{}'.format(len(train_data_df_2),len(train_data_df_1),len(train_data_df_0)))
    if data_type == 'model_4':
        for index,row in train_result_df.iterrows():
            if '0' in row['标签'].split(','):
                if len(set(row['标签'].split(','))) == 2:
                    if row['标签'].split(',')[1]  != '0':
                        train_data_process.append([row['句子'],'0'])
                    else:
                        train_data_process.append([row['句子'],'1'])
                else:
                    train_data_process.append([row['句子'],'0'])
            else:
                if row['标签'].split(',')[0] < row['标签'].split(',')[1]:
                    train_data_process.append([row['句子'],'0'])
                elif row['标签'].split(',')[0] > row['标签'].split(',')[1]:
                    train_data_process.append([row['句子'],'1'])  
                else:
                    train_data_process.append([row['句子'],'0'])
        for index,row in dev_result_df.iterrows():
            if '0' in row['标签'].split(','):
                if len(set(row['标签'].split(','))) == 2:
                    if row['标签'].split(',')[1]  != '0':
                        dev_data_process.append([row['句子'],'0'])
                    else:
                        dev_data_process.append([row['句子'],'1'])
                else:
                    dev_data_process.append([row['句子'],'0'])
            else:
                if row['标签'].split(',')[0] < row['标签'].split(',')[1]:
                    dev_data_process.append([row['句子'],'0'])
                elif row['标签'].split(',')[0] > row['标签'].split(',')[1]:
                    dev_data_process.append([row['句子'],'1'])  
                else:
                    dev_data_process.append([row['句子'],'0'])
        for index,row in test_data_df.iterrows():
            if '0' in row['标签'].split(','):
                if len(set(row['标签'].split(','))) == 2:
                    if row['标签'].split(',')[1]  != '0':
                        test_data_process.append([row['句子'],'0'])
                    else:
                        test_data_process.append([row['句子'],'1'])
                else:
                    test_data_process.append([row['句子'],'0'])
            else:
                if row['标签'].split(',')[0] < row['标签'].split(',')[1]:
                    test_data_process.append([row['句子'],'0'])
                elif row['标签'].split(',')[0] > row['标签'].split(',')[1]:
                    test_data_process.append([row['句子'],'1'])  
                else:
                    test_data_process.append([row['句子'],'0'])
        train_data_df = pd.DataFrame(train_data_process,columns=['sentence','label']).drop_duplicates().reset_index(drop=True)
        train_data_df_0 = train_data_df[train_data_df['label']=='0']
        train_data_df_1 = train_data_df[train_data_df['label']=='1']
        train_data_df_2 = train_data_df[train_data_df['label']=='2']
        print('model_4, 标签2：{}，标签1:{},标签0:{}'.format(len(train_data_df_2),len(train_data_df_1),len(train_data_df_0)))
    if data_type == 'model_5': 
        for index,row in train_result_df.iterrows():
            if '0' in row['标签'].split(','):
                if len(set(row['标签'].split(','))) == 2:
                    if row['标签'].split(',')[1]  != '0':
                        train_data_process.append([row['句子'],'0'])
                    else:
                        train_data_process.append([row['句子'],'1'])
                else:
                    train_data_process.append([row['句子'],'2'])
            else:
                if row['标签'].split(',')[0] < row['标签'].split(',')[1]:
                    train_data_process.append([row['句子'],'0'])
                elif row['标签'].split(',')[0] > row['标签'].split(',')[1]:
                    train_data_process.append([row['句子'],'1'])  
                else:
                    train_data_process.append([row['句子'],'2'])
        for index,row in dev_result_df.iterrows():
            if '0' in row['标签'].split(','):
                if len(set(row['标签'].split(','))) == 2:
                    if row['标签'].split(',')[1]  != '0':
                        dev_data_process.append([row['句子'],'0'])
                    else:
                        dev_data_process.append([row['句子'],'1'])
                else:
                    dev_data_process.append([row['句子'],'2'])
            else:
                if row['标签'].split(',')[0] < row['标签'].split(',')[1]:
                    dev_data_process.append([row['句子'],'0'])
                elif row['标签'].split(',')[0] > row['标签'].split(',')[1]:
                    dev_data_process.append([row['句子'],'1'])  
                else:
                    dev_data_process.append([row['句子'],'2'])
        for index,row in test_data_df.iterrows():
            if '0' in row['标签'].split(','):
                if len(set(row['标签'].split(','))) == 2:
                    if row['标签'].split(',')[1]  != '0':
                        test_data_process.append([row['句子'],'0'])
                    else:
                        test_data_process.append([row['句子'],'1'])
                else:
                    test_data_process.append([row['句子'],'2'])
            else:
                if row['标签'].split(',')[0] < row['标签'].split(',')[1]:
                    test_data_process.append([row['句子'],'0'])
                elif row['标签'].split(',')[0] > row['标签'].split(',')[1]:
                    test_data_process.append([row['句子'],'1'])  
                else:
                    test_data_process.append([row['句子'],'2'])
        train_data_df = pd.DataFrame(train_data_process,columns=['sentence','label']).drop_duplicates().reset_index(drop=True)
        train_data_df_0 = train_data_df[train_data_df['label']=='0']
        train_data_df_1 = train_data_df[train_data_df['label']=='1']
        train_data_df_2 = train_data_df[train_data_df['label']=='2']
        print('model_5 标签2：{}，标签1:{},标签0:{}'.format(len(train_data_df_2),len(train_data_df_1),len(train_data_df_0)))
    train_data_shuf = shuffle(train_data_df)
    f_train = open(train_data_path,'w')
    f_dev = open(dev_data_path,'w')
    f_test = open(test_data_path,'w')
    for index,row in train_data_shuf.iterrows():
        f_train.write(row['sentence'] + '\t' + row['label'] + '\n')
    for row in dev_data_process:
        f_dev.write(row[0] + '\t' + row[1] + '\n') 
    for row in test_data_process:
        f_test.write(row[0] + '\t' + row[1] + '\n')

In [17]:
preprocess_data('model_1')
preprocess_data('model_2')
preprocess_data('model_3')
preprocess_data('model_4')
preprocess_data('model_5')

model_1,标签1:178150,标签0:139905
model_2,标签1:178150,标签0:156443
model_3, 标签2:146890,标签1:178150,标签0:9553
model_4, 标签2：0，标签1:185440,标签0:149153
model_5 标签2：136668，标签1:185440,标签0:12485


In [109]:
results = pd.DataFrame(result,columns=['句子','标签"1"对应的关键词','标签“0”对应的关键词','标签'])

In [112]:
results.to_excel('./安徽省安庆市2007标签处理结果.xlsx',index=None)